## (A) Neural Matrix Factorizarion Steps -  NeuMF Paper

1.   Usually given an "object 1 by object 2" observation matrix M with explicit feedback data.

---

2.   Two learning approaches can be adopted: 

*   Pointwise learning: follows a regression framework by minimizing the squred loss between *yPredicted* and *yObserved*.
*   Pairwise learning: this ranks observed entries higher relative to unobserved entries, and maximizes the margin between them.

---

3. Electing pointwise learning, the training phase involves searching for the optimal parameters that minimizes this squared loss over a reduced (k-dimensional) feature space.

---

*   Predictions can then be made for the unobserved entries.
*   In essence, both the observed and unobserved entries have been approximated by a non-linear function, which presumably improves upon linear(dot product) approximation.

---

What's often done(and adopted by the paper) is to restructure the problem to make use of implicit data (more easy to collect compared to explicit data)

So in this case, the observation matrix M is converted into a binary interaction matrix P. The training prediction processes are similar as in the case of learning with M.

---
---

## (B) Recovering the gene expression matrix(normalized) from LIGER ##

1. Pass the downsampled control and interferon-stimulated PBMCs to the LIGER function.

2. The LIGER function returns a normalized **H1, H2**, **V1, V2**, and **W**(shared) by performing non-negative matrix factorization.

3. These matrices can be used to recover a dense representation of the original expression matrices.

4. ** The "cell paper" then achieves integrative clustering by buildng a shared neighbourhood graph following the five steps of "Joint clustering and factor normalization" under the STAR methods.

---
---

## (C) Self Implementation

### Get data:
1. A first option to getting the gene expression matrices is to recover them from the LIGER output.



  *   The recovered matrices are a dense representation of the original expression matrices
  *  Once the data is obtained this way, NeuMF cannot be applied for the simple reason that there are presumably no negative(unobserved/missing) instances.

2. A second option will be to get the raw representation. This option will allow the application of NeuMF.

---
### Implementation Steps:

1. Create a LIGER object with the raw data (stim & ctrl datasets)
  -  This allows the recovery of a sparse gene by cell matrix.
  - Cells not expressing any genes and genes not expressed in any cell are removed.
  - Any remaining 0 is thus a "true" missing expression.
  - Two sparse matrix representations will result; "ctr_sparse" and "stim_sparse"

2. Feed the two sparse representations separately through the network. 
  - This will produce two dense representations of the sparse versions.
  - The NeuMF will leverage any existing non-linear relationship between the cells and genes whiles maintaining the usual linear operations for better predictions.

3. Using these two dense representations create a LIGER object with placeholders for H1, H2, V1, V2 and the shared matrix W.

4. Scale and normalize the values of the resulting object. This ensures each gene has the same variance and also accounts for varying sequencing depths.

5. Run the NMF algorithm from LIGER to get values for the matrices in (3)

6. Build the shared neighbourhood graph and carry out the clustering. Implementation can be done with the LIGER package.

---
---

## (D) Why this gives better clusters

    ** will denote the paper implementation
    *** will denote the corresponding implemntation in NeuMF
---

1. ### Factorization Method:

---
  ** -- Finds the matrices (of reduced dimensions) in (C)(3) by minimizing the penealized frobenius norm squared error between the observed and estimated matrices.

  *** -- Finds some two lower dimensional matrics say U & V, that densely approximates the observed expression matrix by minimizing the MSE.

  * The two loss functions are similar and differ only by a transformation.

---

2. ### Optimization Algorithm:

---

  **  -- Uses Block Cordinate Descent. It iteratively minimizes the factorization objective by making use of profiling. Convergence (local min) is guaranteed.

  *** -- Uses Stochastic Gradient Descent. Simultaneously update model network parameters to minimze the loss function. Convergence is not guaranteed but saddle points in deep networks have shown to produce optimal functions than their shallow counterparts (BCD can be formulated as such).

---

3. ### Prediction Function:

---

  **  -- Basically a linear approximation.

  *** -- Introduces non-linearities via the activation fucntions.

---

* For any cell gene expression matrix, NeuMF will always perform atleast as well as NMF Liger implementation. Performance will be indistninguishable if only linear (unlikely) relationship is present.

* For the same factor specification k, performing Liger NMF on a dense output from NeuMF will not have any impact.

* The above is why it makes sense to create a LIGER object with the matrices from NeuMF and cluster based on LIGERs NNB implementation.

---

## **Code**

In [ ]:
# load packages
import csv
import math
import time
import numpy as np
import scipy as sc
import pandas as pd
from tqdm import tqdm
import urllib.request as urllib
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from sklearn.manifold import TSNE
import sklearn.cluster as cluster
from sklearn.utils import shuffle
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

Instructions for updating:
non-resource variables are not supported in the long term


## **(1) Load and Process Data**

In [ ]:
# load data
url = 'https://sixtusdakurah.com/projects/liger/stim_sparse_dfp.csv'
ctrl_sparse = pd.read_csv(url, sep=",", header=0)
ctrl_sparse = ctrl_sparse.rename(columns={'Unnamed: 0': 'gene'})

In [ ]:
# check dimensions and get brief view
ctrl_sparse.head()

,gene,stimAGGACACTCATGGT-1,stimCCCTTACTTTGCGA-1,stimGCTACCTGTGGTCA-1,stimTCCTAATGTCTCTA-1,stimTAGCCCTGACCTCC-1,stimGGTCTAGAAGTGTC-1,stimTGACGAACTACGCA-1,stimTCATTCGATTTACC-1,stimAGGTACTGTTCCAT-1,...,stimCCGCTATGGTCCTC-1,stimATCTTGACGCTGAT-1,stimCGGTCACTAGCACT-1,stimAATCTCTGGTATGC-1,stimCGGTACCTAGATGA-1,stimTTGGTACTTTCCCG-1,stimGGAGGCCTTTTACC-1,stimTGCAGATGACCGAT-1,stimACTCCTCTACGGAG-1,stimGCATTGGATATGCG-1
0,AL627309.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,RP11.206L10.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RP11.206L10.9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LINC00115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NOC2L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print("Shape of Ctrl: ", ctrl_sparse.shape)

Shape of Ctrl:  (14578, 3001)


In [ ]:
ctr_E_df = ctrl_sparse#.iloc[0:6000, 0:3001]
# convert to long format
ctr_E_df_long = pd.melt(ctr_E_df, id_vars=['gene'],var_name='cell', value_name='expression')

In [ ]:
ctr_E_df.head()

,gene,stimAGGACACTCATGGT-1,stimCCCTTACTTTGCGA-1,stimGCTACCTGTGGTCA-1,stimTCCTAATGTCTCTA-1,stimTAGCCCTGACCTCC-1,stimGGTCTAGAAGTGTC-1,stimTGACGAACTACGCA-1,stimTCATTCGATTTACC-1,stimAGGTACTGTTCCAT-1,...,stimCCGCTATGGTCCTC-1,stimATCTTGACGCTGAT-1,stimCGGTCACTAGCACT-1,stimAATCTCTGGTATGC-1,stimCGGTACCTAGATGA-1,stimTTGGTACTTTCCCG-1,stimGGAGGCCTTTTACC-1,stimTGCAGATGACCGAT-1,stimACTCCTCTACGGAG-1,stimGCATTGGATATGCG-1
0,AL627309.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,RP11.206L10.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RP11.206L10.9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LINC00115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NOC2L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ctr_E_df_long.head()

,gene,cell,expression
0,AL627309.1,stimAGGACACTCATGGT-1,0
1,RP11.206L10.2,stimAGGACACTCATGGT-1,0
2,RP11.206L10.9,stimAGGACACTCATGGT-1,0
3,LINC00115,stimAGGACACTCATGGT-1,0
4,NOC2L,stimAGGACACTCATGGT-1,0


In [ ]:
(ctr_E_df_long == 0).astype(int).sum(axis=0)

gene                 0
cell                 0
expression    41894690
dtype: int64

In [ ]:
print("Unique genes: ", len((ctr_E_df_long['gene'].astype("category").cat.codes).drop_duplicates()))#.sort_values()
print("Unique cells: ",len((ctr_E_df_long['cell'].astype("category").cat.codes).drop_duplicates()))

Unique genes:  14578
Unique cells:  3000


## **(2) Build Helper Functions**

In [ ]:
# process data set
def process_dataset(df):

    # Convert cells names into numerical IDs
    df['cell_id'] = df['cell'].astype("category").cat.codes
    df['gene_id'] = df['gene'].astype("category").cat.codes

    
    gene_lookup = df[['gene_id', 'gene']].drop_duplicates()
    gene_lookup['gene_id'] = gene_lookup.gene_id.astype(str)

    # Grab the columns we need in the order we need them.
    df = df[['cell_id', 'gene_id', 'expression']]

    
    #df_train, df_test = train_test_split(df) # 80 20
    df_train, df_test = df, ''


    
    cells = list(np.sort(df.cell_id.unique()))
    genes = list(np.sort(df.gene_id.unique()))

    
    rows = df_train.cell_id.astype(int)
    cols = df_train.gene_id.astype(int)

    values = list(df_train.expression)

    # Get all user ids and item ids.
    cids = np.array(rows.tolist())
    gids = np.array(cols.tolist())

    # Sample 100 negative interactions for each cell in our test data
    df_neg = '' #get_negatives(cids, gids, genes, df_test)

    return cids, gids, df_train, df_test, df_neg, cells, genes, gene_lookup, values

In [ ]:
# sample a couple of negatives for each positive label
def get_negatives(cids, gids, genes, df_test):
    
    negativeList = []
    test_c = df_test['cell_id'].values.tolist()
    test_g = df_test['gene_id'].values.tolist()

    test_expression_ids = list(zip(test_c, test_g))
    zipped = set(zip(cids, gids))
    #print(len(genes))

    for (c, g) in test_expression_ids:
        negatives = []
        negatives.append((c, g))
        for t in range(10):# increase for better accuracy
            j = np.random.randint(len(genes)) # Get random gene id.
            while (c, j) in zipped: # Check if there is an interaction
                j = np.random.randint(len(genes)) # If yes, generate a new gene id
            negatives.append(j) # Once a negative interaction is found we add it.
            #print("J value is", j)
            #print(negatives)
        negativeList.append(negatives)

    df_neg = pd.DataFrame(negativeList)

    return df_neg

In [ ]:
# mask the first gene to be used for testing
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    
    return result

# split into train and test set   
def train_test_split(df):

    df_test = df.copy(deep=True)
    df_train = df.copy(deep=True)

    df_test = df_test.groupby(['cell_id']).first()
    df_test['cell_id'] = df_test.index
    df_test = df_test[['cell_id', 'gene_id', 'expression']]
    df_test.index.name = None

    mask = df.groupby(['cell_id'])['cell_id'].transform(mask_first).astype(bool)
    df_train = df.loc[mask]

    return df_train, df_test

# combine mask and train test split
def get_train_instances():
    
    cell_input, gene_input, labels = [],[],[]
    zipped = set(zip(cids, gids))
    #progress = tqdm(total=len(zipped))
    #tracker = 0
    for (c, g) in zip(cids, gids):
        # Add our positive interaction
        cell_input.append(c)
        gene_input.append(g)
        labels.append((df_train[(df_train.cell_id==c)&(df_train.gene_id==g)]).expression.values[0])
        #labels.append(1)

        # Sample a number of random negative interactions
        for t in range(num_neg):
            j = np.random.randint(len(genes))
            #j = j if j!=32 else np.random.randint(len(genes)) # chainging to more than 1 
            while (c, j) in zipped:
                j = np.random.randint(len(genes))
                #j = rv+1 if rv==0 else rv
            #print("gene value: ", j, " cell value: ", c)
            if j!=df_test.gene_id[0]:
                
                cell_input.append(c)
                gene_input.append(j)
              #print("gene value: ", j, " cell value: ", c)
              #labels.append(0)
                labels.append((df_train[(df_train.cell_id==c)&(df_train.gene_id==j)]).expression.values[0])
        #progress.update(1)
        #progress.set_description('Sampled Training Instance' + str(tracker+1)) 
        #tracker+=1
    #progress.close()
    return cell_input, gene_input, labels

# for faster training
def random_mini_batches(C, G, L, mini_batch_size=20):

    mini_batches = []

    shuffled_C, shuffled_G, shuffled_L = shuffle(C, G, L, random_state=0)

    num_complete_batches = int(math.floor(len(C)/mini_batch_size))
    for k in range(0, num_complete_batches):
        mini_batch_C = shuffled_C[k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_G = shuffled_G[k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_L = shuffled_L[k * mini_batch_size : k * mini_batch_size + mini_batch_size]

        mini_batch = (mini_batch_C, mini_batch_G, mini_batch_L)
        mini_batches.append(mini_batch)

    if len(C) % mini_batch_size != 0:
        mini_batch_C = shuffled_C[num_complete_batches * mini_batch_size: len(C)]
        mini_batch_G = shuffled_G[num_complete_batches * mini_batch_size: len(C)]
        mini_batch_L = shuffled_L[num_complete_batches * mini_batch_size: len(C)]

        mini_batch = (mini_batch_C, mini_batch_G, mini_batch_L)
        mini_batches.append(mini_batch)

    return mini_batches

In [ ]:
# evaluation
def get_hits(k_ranked, holdout):
    for gene in k_ranked:
        if gene == holdout:
            return 1
    return 0

def eval_rating(idx, test_expression, test_negatives, K):
    # test_expression = test_expression_ids
    map_gene_score = {}

    
    genes = test_negatives[idx]

    
    cell_idx = test_expression[idx][0]

    
    holdout = test_expression[idx][1]
    print("Holdout: ", holdout)

    
    genes.append(holdout)

    
    predict_cell = np.full(len(genes), cell_idx, dtype='int32').reshape(-1,1)
    print("Predict cell: ", predict_cell)
    np_genes = np.array(genes).reshape(-1,1)
    print("Genes: ", genes)

    
    predictions = session.run([output_layer], feed_dict={cell: predict_cell, gene: np_genes})
    print("Predictions: ", predictions)
    
    predictions = predictions[0].flatten().tolist()

    
    for i in range(len(genes)):
        current_gene = genes[i]
        map_gene_score[current_gene] = predictions[i]

    
    k_ranked = heapq.nlargest(K, map_gene_score, key=map_gene_score.get)
    print("K Ranked: ", k_ranked)

       
    hits = get_hits(k_ranked, holdout)

    return hits

# 
def evaluate(df_neg, K=10):

    hits = []

    test_c = df_test['cell_id'].values.tolist()
    test_g = df_test['gene_id'].values.tolist()

    test_expression_ids = list(zip(test_c, test_g))

    df_neg = df_neg.drop(df_neg.columns[0], axis=1)
    test_negatives = df_neg.values.tolist()
    #len(test_expression)-2
    for idx in range(len(test_expression_ids)):
        # For each idx, call eval_one_rating
        hitrate = eval_rating(idx, test_expression_ids, test_negatives, K)
        hits.append(hitrate)

    return hits


def poisson_loss(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    f1 = tf.multiply(y_true, tf.log(tf.add(y_pred, 1e-10)))
    nice_bound = tf.add(tf.lgamma(y_pred), 1)
    fbound = tf.subtract(f1, nice_bound)
    return tf.reduce_mean(tf.square(tf.subtract(y_pred, fbound)))

In [ ]:
def make_recommendation(cell_ids=None, gene_ids = None, top=None):
    # make recommendations for all
    df = pd.DataFrame()
    df["Gene"] = (ctr_E_df_long.gene_id).unique()
    for cell_idx in np.sort((ctr_E_df_long.cell_id).unique()):
        # get the genes for the given cell
        genes = ((ctr_E_df_long[ctr_E_df_long.cell_id==cell_idx]).gene_id).values
        #cell_ls = [cell_idx]# * len(genes)
        # create a full gene prediction for a cell
        predict_cell = np.full(len(genes), cell_idx, dtype='int32').reshape(-1,1)
        np_genes = np.array(genes).reshape(-1,1)
        
        #run with the given session
        predictions = session.run([output_layer], feed_dict={cell: predict_cell, gene: np_genes})
        #print("Predictions: ", predictions)

        predictions = predictions[0].flatten().tolist()
        df[cell_idx] = predictions
 
    return df

# try basic k-means clustering
# def kMeans_clustering(k=10):
    
    
#       kmeans_labels = cluster.KMeans(n_clusters=k).fit_predict(mlp_df)
#       standard_embedding = umap.UMAP(random_state=42).fit_transform(mlp_df)
#       newdf = pd.DataFrame(standard_embedding, columns = ["x1", "x2"])
#       newdf["cluster"] = kmeans_labels
#       # make the plot
#       size = 80
#       plt.figure(figsize=(16,10))
#       sc = plt.scatter(newdf['x1'], newdf['x2'], s=size, c=newdf['cluster'], edgecolors='none')

#       lp = lambda i: plt.plot([],color=sc.cmap(sc.norm(i)), ms=np.sqrt(size), mec="none",
#                               label="Cluster {:g}".format(i), ls="", marker="o")[0]
#       handles = [lp(i) for i in np.unique(newdf["cluster"])]
#       plt.legend(handles=handles)
#       plt.xlabel("Umap 1")
#       plt.ylabel("Umap 2")
#       plt.show()

In [ ]:
cids, gids, df_train, df_test, df_neg, cells, genes, gene_lookup, values = process_dataset(ctr_E_df_long)

In [ ]:
print("Unique train genes: ", len((df_train['gene_id']).drop_duplicates()))
print("Unique train cells: ", len((df_train['cell_id']).drop_duplicates()))
#print("Unique test genes: ", len((df_test['gene_id']).drop_duplicates()))
#print("Unique test cells: ",len((df_test['cell_id']).drop_duplicates()))

Unique train genes:  14578
Unique train cells:  3000


In [ ]:
df_train.head()

,cell_id,gene_id,expression
0,553,594,0
1,553,9998,0
2,553,9999,0
3,553,6366,0
4,553,7804,0


In [ ]:
df_train.isnull().values.any()

False

In [ ]:
df_train.shape

(43734000, 3)

In [ ]:
df_train.shape[0]/256

170835.9375

### **(3) Set Training Parameters**

In [ ]:
# define learning parameters
num_neg = 4
epochs = 20
batch_size = 16 # 32 fro stim, 64 for ctrl
learning_rate = 0.001
latent_features = 20

In [ ]:
# get train instances
cell_input, gene_input, labels = cids, gids, values#get_train_instances()

In [ ]:
#sum(labels)

### (4) **Build & Train MLP Network**

In [ ]:
graph = tf.Graph() # tensorflow term for building a pipeline

with graph.as_default():

    # define input placeholders for cell, gene and count=label.
    cell = tf.placeholder(tf.int32, shape=(None, 1))
    gene = tf.placeholder(tf.int32, shape=(None, 1))
    label = tf.placeholder(tf.int32, shape=(None, 1))

    # cell feature embedding
    c_var = tf.Variable(tf.random_uniform([len(cells), latent_features], minval= 0), name='cell_embedding')
    cell_embedding = tf.nn.embedding_lookup(c_var, cell) # for each cell id, return a vector of length 20

    # gene feature embedding
    g_var = tf.Variable(tf.random_uniform([len(genes), latent_features], minval= 0), name='gene_embedding')
    gene_embedding = tf.nn.embedding_lookup(g_var, gene) # for each gene id, return a vector of length 20

    # Flatten our cell and gene embeddings.
    cell_embedding = tf.keras.layers.Flatten()(cell_embedding)
    gene_embedding = tf.keras.layers.Flatten()(gene_embedding)

    # concatenate the two embedding vectors
    concatenated = tf.keras.layers.concatenate([cell_embedding, gene_embedding])

    
    # add a first dropout layer.
    dropout = tf.keras.layers.Dropout(0.2)(concatenated)

    # add four hidden layers along with batch
    # normalization and dropouts. use relu as the activation function.
    layer_1 = tf.keras.layers.Dense(64, activation='relu', name='layer1')(dropout)
    batch_norm1 = tf.keras.layers.BatchNormalization(name='batch_norm1')(layer_1)
    dropout1 = tf.keras.layers.Dropout(0.2, name='dropout1')(batch_norm1)

    layer_2 = tf.keras.layers.Dense(32, activation='linear', name='layer2')(layer_1)
    batch_norm2 = tf.keras.layers.BatchNormalization(name='batch_norm1')(layer_2)
    dropout2 = tf.keras.layers.Dropout(0.2, name='dropout1')(batch_norm2)

    layer_3 = tf.keras.layers.Dense(16, activation='relu', name='layer3')(layer_2)
    layer_4 = tf.keras.layers.Dense(8, activation='exponential', name='layer4')(layer_3) # make linear

    # final single neuron output layer.
    output_layer = tf.keras.layers.Dense(1,
            kernel_initializer="lecun_uniform",
            name='output_layer')(layer_4)

    # our loss function as mse.
    labels = tf.cast(label, tf.float32)
    #print(labels)
    logits = output_layer
    #tf.print(logits)
    #loss =  poisson_loss(labels, logits) #tf.nn.log_poisson_loss(labels, logits)
    loss = poisson_loss(labels, logits)
    #print(loss)

    # train using the Adam optimizer to minimize loss.
    opt = tf.train.AdamOptimizer(learning_rate = learning_rate, epsilon=1)
    step = opt.minimize(loss)

    # initialize all tensorflow variables.
    init = tf.global_variables_initializer()

session = tf.Session(config=None, graph=graph)
session.run(init)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# for epoch in range(epochs):

#     # Get our training input.
#     cell_input, gene_input, labels = cids, gids, values  #get_train_instances()

#     # Generate a list of minibatches.
#     minibatches = random_mini_batches(cell_input, gene_input, labels)

#     # This has noting to do with tensorflow but gives
#     # us a nice progress bar for the training
#     progress = tqdm(total=len(minibatches))

#     # Loop over each batch and feed our cells, genes and labels
#     # into our graph. 
#     for minibatch in minibatches:
#         feed_dict = {cell: np.array(minibatch[0]).reshape(-1,1),
#                     gene: np.array(minibatch[1]).reshape(-1,1),
#                     label: np.array(minibatch[2]).reshape(-1,1)}
   
#         # Execute the graph.
#         _, l = session.run([step, loss], feed_dict)
#         # Update the progress
#         progress.update(1)
#         progress.set_description('Epoch: %d - Loss: %.8f' % (epoch+1, l))

#     progress.close()

In [ ]:
# mlp_df = make_recommendation()
# #mlp_df.drop('Gene', axis=1, inplace=True)
# mlp_df.head()

In [ ]:
# # try basic clustering
# kMeans_clustering()

### **(5) Build & Train GMF Network**

In [ ]:
graph = tf.Graph()

with graph.as_default():

    cell = tf.placeholder(tf.int32, shape=(None, 1))
    gene = tf.placeholder(tf.int32, shape=(None, 1))
    label = tf.placeholder(tf.int32, shape=(None, 1))

    
    c_var = tf.Variable(tf.random_uniform([len(cells), latent_features],
                                         minval =0.0), name='cell_embedding')
    cell_embedding = tf.nn.embedding_lookup(c_var, cell)


    g_var = tf.Variable(tf.random_uniform([len(genes), latent_features],
                                         minval=0.0), name='gene_embedding')
    gene_embedding = tf.nn.embedding_lookup(g_var, gene)
    
    # flatten the embeddings 
    cell_embedding = tf.keras.layers.Flatten()(cell_embedding)
    gene_embedding = tf.keras.layers.Flatten()(gene_embedding)

    # multiplying our cell and gene latent space vectors together 
    prediction_matrix = tf.multiply(cell_embedding, gene_embedding)

    
    output_layer = tf.keras.layers.Dense(1, 
            kernel_initializer="lecun_uniform",
            name='output_layer')(prediction_matrix)

    # loss function as mse. 
    labels = tf.cast(label, tf.float32)
    loss = tf.reduce_mean(tf.square(tf.subtract(labels, output_layer)))
    
    # using the Adam optimizer to minimize loss.
    opt = tf.train.AdamOptimizer(learning_rate = learning_rate)
    step = opt.minimize(loss)

    # initialize all tensorflow variables.
    init = tf.global_variables_initializer()

session = tf.Session(config=None, graph=graph)
session.run(init)

In [ ]:
# for epoch in range(epochs):

#     # Get our training input.
#     cell_input, gene_input, labels = get_train_instances()

#     # Generate a list of minibatches.
#     minibatches = random_mini_batches(cell_input, gene_input, labels)

#     # This has noting to do with tensorflow but gives
#     # us a nice progress bar for the training
#     progress = tqdm(total=len(minibatches))

#     # Loop over each batch and feed our cells, genes and labels
#     # into our graph. 
#     for minibatch in minibatches:
#         feed_dict = {cell: np.array(minibatch[0]).reshape(-1,1),
#                     gene: np.array(minibatch[1]).reshape(-1,1),
#                     label: np.array(minibatch[2]).reshape(-1,1)}
   
#         # Execute the graph.
#         _, l = session.run([step, loss], feed_dict)

#         # Update the progress
#         progress.update(1)
#         progress.set_description('Epoch: %d - Loss: %.3f' % (epoch+1, l))

#     progress.close()

In [ ]:
# mlp_df = make_recommendation()
# mlp_df.drop('Gene', axis=1, inplace=True)
# mlp_df.head()

In [ ]:
# # try basic clustering
# kMeans_clustering()

### **(6) Build & Train Combined NeuMF**

In [ ]:
graph = tf.Graph()

with graph.as_default():

    cell = tf.placeholder(tf.int32, shape=(None, 1))
    gene = tf.placeholder(tf.int32, shape=(None, 1))
    label = tf.placeholder(tf.int32, shape=(None, 1))

    
    mlp_c_var = tf.Variable(tf.random_uniform([len(cells), latent_features], minval=0), name='mlp_cell_embedding')
    mlp_cell_embedding = tf.nn.embedding_lookup(mlp_c_var, cell)

    
    mlp_g_var = tf.Variable(tf.random_uniform([len(genes), latent_features], minval=0), name='mlp_gene_embedding')
    mlp_gene_embedding = tf.nn.embedding_lookup(mlp_g_var, gene)

    
    gmf_c_var = tf.Variable(tf.random_uniform([len(cells), latent_features], minval=0), name='gmf_cell_embedding')
    gmf_cell_embedding = tf.nn.embedding_lookup(gmf_c_var, cell)

    # gene embedding for GMF
    gmf_g_var = tf.Variable(tf.random_uniform([len(genes), latent_features], minval=0), name='gmf_item_embedding')
    gmf_gene_embedding = tf.nn.embedding_lookup(gmf_g_var, gene)

    # flatten gmf embedding
    gmf_cell_embed = tf.keras.layers.Flatten()(gmf_cell_embedding)
    gmf_gene_embed = tf.keras.layers.Flatten()(gmf_gene_embedding)
    gmf_matrix = tf.multiply(gmf_cell_embed, gmf_gene_embed)

    # flatten mlp embedding
    mlp_cell_embed = tf.keras.layers.Flatten()(mlp_cell_embedding)
    mlp_gene_embed = tf.keras.layers.Flatten()(mlp_gene_embedding)
    mlp_concat = tf.keras.layers.concatenate([mlp_cell_embed, mlp_gene_embed])

    mlp_dropout = tf.keras.layers.Dropout(0.2)(mlp_concat)

    mlp_layer_1 = tf.keras.layers.Dense(64, activation='relu', name='layer1')(mlp_dropout)
    mlp_batch_norm1 = tf.keras.layers.BatchNormalization(name='batch_norm1')(mlp_layer_1)
    mlp_dropout1 = tf.keras.layers.Dropout(0.2, name='dropout1')(mlp_batch_norm1)

    mlp_layer_2 = tf.keras.layers.Dense(32, activation='relu', name='layer2')(mlp_dropout1)
    mlp_batch_norm2 = tf.keras.layers.BatchNormalization(name='batch_norm1')(mlp_layer_2)
    mlp_dropout2 = tf.keras.layers.Dropout(0.2, name='dropout1')(mlp_batch_norm2)

    mlp_layer_3 = tf.keras.layers.Dense(16, activation='relu', name='layer3')(mlp_dropout2)
    mlp_layer_4 = tf.keras.layers.Dense(8, activation='exponential', name='layer4')(mlp_layer_3)

    # We merge the two networks together
    merged_vector = tf.keras.layers.concatenate([gmf_matrix, mlp_layer_4])

    # Our final single neuron output layer. 
    output_layer = tf.keras.layers.Dense(1,
            kernel_initializer="lecun_uniform",
            name='output_layer')(merged_vector)

    # Our loss function as mse. 
    labels = tf.cast(label, tf.float32)
    loss = tf.reduce_mean(tf.square(tf.subtract(
                labels,
                output_layer)))
    #loss = poisson_loss(labels, output_layer)

    # Train using the Adam optimizer to minimize our loss.
    opt = tf.train.AdamOptimizer(learning_rate = learning_rate, epsilon=1)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()


session = tf.Session(config=None, graph=graph)
session.run(init)

In [ ]:
for epoch in range(epochs):

    # Get our training input.
    cell_input, gene_input, labels = cids, gids, values #get_train_instances()

    # Generate a list of minibatches.
    minibatches = random_mini_batches(cell_input, gene_input, labels, batch_size)

    # This has noting to do with tensorflow but gives
    # us a nice progress bar for the training
    progress = tqdm(total=len(minibatches))

    # Loop over each batch and feed our cells, genes and labels
    # into our graph. 
    for minibatch in minibatches:
        feed_dict = {cell: np.array(minibatch[0]).reshape(-1,1),
                    gene: np.array(minibatch[1]).reshape(-1,1),
                    label: np.array(minibatch[2]).reshape(-1,1)}
   
        # Execute the graph.
        _, l = session.run([step, loss], feed_dict)
        # Update the progress
        progress.update(1)
        progress.set_description('Epoch: %d - Loss: %.3f' % (epoch+1, l))

    progress.close()

Epoch: 4 - Loss: 0.004:  16%|█▌        | 442551/2733375 [22:39<1:54:45, 332.68it/s]     

In [ ]:
mlp_df = make_recommendation()
#mlp_df.drop('Gene', axis=1, inplace=True) #uncomment to cluster but comment out when writing out to csv file.
mlp_df.head()

In [ ]:
# try basic clustering
#kMeans_clustering(10)

### **(7) Save Dense Matrix to File**

In [ ]:
mlp_df.to_csv("stim_dense_shortAll3.csv", index = False)